In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.tools import tool, ToolRuntime


@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]


@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [ ]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware


class EmailState(AgentState):
    email: str


agent = create_agent(
    model="gpt-5-nano",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [ ]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response immediately. Send the reply now in the same thread.")],
        "email": "Hi Mark, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, Julii."
    },
    config=config
)

In [ ]:
from pprint import pprint

pprint(response)

In [ ]:
print(response['__interrupt__'])

In [ ]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

## Approve

In [ ]:
from langgraph.types import Command

response = agent.invoke(
    Command(
        resume={"decisions": [{"type": "approve"}]}
    ),
    config=config  # Same thread ID to resume the paused conversation
)

pprint(response)

## Reject

In [ ]:
response = agent.invoke(
    Command(
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Mark."
                }
            ]
        }
    ),
    config=config  # Same thread ID to resume the paused conversation
)

pprint(response)

In [ ]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

## Edit

In [ ]:
response = agent.invoke(
    Command(
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ),
    config=config  # Same thread ID to resume the paused conversation
)

pprint(response)